# Ecosystem services widgets  
This contains two widgets with independent data:
1. Above/below ground organic carbon
2. Commercial fishing restoration

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import fiona

## Above/below ground organic carbon data

In [3]:
#gdb_file =f'../../../data/Typology_and_Restoration_Potential/Data/MOW_Global_Mangrove_Restoration_20190411.gdb'
#all_data = gpd.read_file(gdb_file, driver='FileGDB', layer=0)

data_url = 'https://storage.googleapis.com/mangrove_atlas/widget_data'
all_data = pd.read_csv(f'{data_url}/Vizz_Country_Stats_Dec2022.csv', index_col=0)
all_data.head()

,Name,Total Mangrove Area 2020 (ha),Total Area of Loss (ha),Total Area Restorable (ha),OVERALL_SRE,Total_Fish,Fish,Crab,Bivalve,Shrimp,Total AGC (mtCO2e) from restored areas,Total SOC (mtCO2e) from restored areas,Proportion_Restorable,Extreme_Weather,Non_Productive,Commodities,Driver
1,American Samoa,32.004317,0.594951,0.442066,0.830489,9.129377e+03,9081.125331,4.825178e+01,0.0,0.0,82.919195,144.531066,1.38,0.000619,0.000594,0.003208,Commodities
2,Angola,28319.919690,1546.820441,856.958013,0.692000,4.553915e+04,45539.145920,0.000000e+00,0.0,0.0,110509.432500,234265.161100,3.03,0.254498,8.113579,0.000000,Non-Productive Conversion
3,Anguilla,3.836882,1.571377,1.571377,0.620000,2.378690e+04,10165.451040,1.362145e+04,0.0,0.0,11.015105,461.861506,40.95,0.000000,0.015714,0.000000,Non-Productive Conversion
4,Antigua and Barbuda,871.859248,107.809050,107.809050,0.632000,2.045093e+06,873980.752300,1.171113e+06,0.0,0.0,2714.655158,28326.766570,12.37,0.000000,1.078090,0.000000,Non-Productive Conversion
5,Aruba,47.371025,8.345105,5.643541,0.597168,8.612742e+04,36806.980660,4.932044e+04,0.0,0.0,185.684321,1142.345668,11.91,0.016808,0.034860,0.004767,Non-Productive Conversion


In [6]:
carbon_data = all_data[['Name','Total AGC (mtCO2e) from restored areas', 'Total SOC (mtCO2e) from restored areas']].copy()
carbon_data.rename(columns={'Name':'Country','Total AGC (mtCO2e) from restored areas':'AGB', 'Total SOC (mtCO2e) from restored areas':'SOC'}, inplace=True)
carbon_data

,Country,AGB,SOC
1,American Samoa,8.291920e+01,1.445311e+02
2,Angola,1.105094e+05,2.342652e+05
3,Anguilla,1.101510e+01,4.618615e+02
4,Antigua and Barbuda,2.714655e+03,2.832677e+04
5,Aruba,1.856843e+02,1.142346e+03
...,...,...,...
118,Vietnam,1.905081e+06,8.665969e+06
119,British Virgin Islands,2.902559e+02,6.823373e+03
120,"Virgin Islands, U.S.",1.661686e+03,1.410356e+04
121,Wallis and Futuna,NaN,NaN


In [7]:
carbon_long = pd.melt(carbon_data, id_vars='Country')
carbon_long.rename(columns={'variable':'indicator'}, inplace=True)
carbon_long

,Country,indicator,value
0,American Samoa,AGB,8.291920e+01
1,Angola,AGB,1.105094e+05
2,Anguilla,AGB,1.101510e+01
3,Antigua and Barbuda,AGB,2.714655e+03
4,Aruba,AGB,1.856843e+02
...,...,...,...
239,Vietnam,SOC,8.665969e+06
240,British Virgin Islands,SOC,6.823373e+03
241,"Virgin Islands, U.S.",SOC,1.410356e+04
242,Wallis and Futuna,SOC,NaN


## Add Locations data  
1) Add ISO codes from locations file
2) Add API location ids

In [11]:
locations_file = 'https://storage.googleapis.com/mangrove_atlas/boundaries/processed/location_final/locations_v3_not_merged_with_old.gpkg'
locations = gpd.read_file(locations_file)
locations = locations[locations['type'] == 'country'][['name', 'iso', 'location_idn']]
locations.head()

,name,iso,location_idn
82,Qatar,QAT,06d2e6f9-bc89-59bf-a0e2-ab804e5db9fd
89,Mayotte,MYT,0750953f-4af9-549b-aeea-329663249a56
118,Vietnam,VNM,09a1ab14-11ad-56ec-8acb-a149e5697abd
132,Grenada,GRD,0b0ecb56-bb8e-5ef1-b8ee-3cdad67fed0e
149,India,IND,0c07ca53-7b17-5650-a2c6-0cc27249a4bd


In [12]:
api_locs = pd.read_csv('https://storage.googleapis.com/mangrove_atlas/widget_data/locations_staging.csv')
api_locs.rename(columns={'location_id': 'location_idn'}, inplace=True)
api_locs.head()

,id,location_idn
0,1563,000bd204-c0fd-510b-a1ad-132a7ef7470d
1,1564,00250a0f-f66d-54a0-b7a3-d80035881cbf
2,1565,0041637b-f6a2-5b89-87ce-850f5c5431b3
3,1566,005b49ef-6b7f-575a-85b3-ff19261a0755
4,1567,00921349-70fb-5a7e-8207-b3157aecc349


In [13]:
country_codes = locations[['name','iso']].copy()
country_codes.head()

,name,iso
82,Qatar,QAT
89,Mayotte,MYT
118,Vietnam,VNM
132,Grenada,GRD
149,India,IND


In [22]:
carbon_long_iso = pd.merge(carbon_long, country_codes, left_on='Country', right_on='name',how='left')
carbon_long_iso

,Country,indicator,value,name,iso
0,American Samoa,AGB,8.291920e+01,American Samoa,ASM
1,Angola,AGB,1.105094e+05,Angola,AGO
2,Anguilla,AGB,1.101510e+01,Anguilla,AIA
3,Antigua and Barbuda,AGB,2.714655e+03,Antigua and Barbuda,ATG
4,Aruba,AGB,1.856843e+02,Aruba,ABW
...,...,...,...,...,...
239,Vietnam,SOC,8.665969e+06,Vietnam,VNM
240,British Virgin Islands,SOC,6.823373e+03,British Virgin Islands,VGB
241,"Virgin Islands, U.S.",SOC,1.410356e+04,"Virgin Islands, U.S.",VIR
242,Wallis and Futuna,SOC,NaN,Wallis and Futuna,WLF


In [23]:
list(carbon_long_iso[carbon_long_iso['iso'].isnull()]['Country'].unique())

['Congo', "Cote d'Ivoire", 'Mexico', 'Sao Tome and Principe']

In [24]:
carbon_long_iso.loc[carbon_long_iso.Country == 'Mexico', 'iso'] = 'MEX'
carbon_long_iso.loc[carbon_long_iso.Country == 'Congo', 'iso'] = 'COG'
carbon_long_iso.loc[carbon_long_iso.Country == 'Sao Tome and Principe', 'iso'] = 'STP'


In [25]:
carbon_long_iso.drop(columns=['name', 'Country'], inplace=True)
carbon_long_iso = carbon_long_iso[carbon_long_iso.iso.notnull()]
carbon_long_iso = carbon_long_iso[carbon_long_iso.value.notnull()]
carbon_long_iso

,indicator,value,iso
0,AGB,8.291920e+01,ASM
1,AGB,1.105094e+05,AGO
2,AGB,1.101510e+01,AIA
3,AGB,2.714655e+03,ATG
4,AGB,1.856843e+02,ABW
...,...,...,...
238,SOC,1.468958e+06,VEN
239,SOC,8.665969e+06,VNM
240,SOC,6.823373e+03,VGB
241,SOC,1.410356e+04,VIR


Add final location ids

In [26]:
df_final = pd.merge(carbon_long_iso, locations, on='iso', how='left')
df_final = df_final.merge(api_locs, on='location_idn', how='left')
df_final.rename(columns={'id':'location_id'}, inplace=True)

df_final

,indicator,value,iso,name,location_idn,location_id
0,AGB,8.291920e+01,ASM,American Samoa,404d005a-797d-5509-91eb-e17ed1069ed6,2346
1,AGB,1.105094e+05,AGO,Angola,27ceab8c-946e-5286-a06f-8bd98ec81f77,2029
2,AGB,1.101510e+01,AIA,Anguilla,1ce4c2e5-8456-5db8-8e34-8bfe86083790,1915
3,AGB,2.714655e+03,ATG,Antigua and Barbuda,7c8d9de5-4c1a-5ed4-838c-05906eaed3f7,3095
4,AGB,1.856843e+02,ABW,Aruba,5d1bd36b-f200-593d-972a-bf69abdd912f,2707
...,...,...,...,...,...,...
213,SOC,1.468958e+06,VEN,Venezuela,a22a14b8-b4dd-55e6-af76-87f1bf68af2a,3582
214,SOC,8.665969e+06,VNM,Vietnam,09a1ab14-11ad-56ec-8acb-a149e5697abd,1681
215,SOC,6.823373e+03,VGB,British Virgin Islands,7802b655-2b5f-5d2b-ab92-ae43ee20c174,3037
216,SOC,1.410356e+04,VIR,"Virgin Islands, U.S.",3fb957bc-db23-5b2e-8f5d-d021133b9414,2339


In [28]:
df_final[~df_final['location_id'].isna()][['location_id', 'indicator', 'value']].to_csv('../../../../data/ecosystem_services_carbon_data_202212.csv', index=False)

In [35]:
carbon_long[carbon_long.Country == 'Mexico']

,Country,indicator,value
69,Mexico,AGB,1.605274e+06
191,Mexico,SOC,2.005485e+07


In [40]:
old_carbon = pd.read_csv('../../../../data/UPDATED_ecosystem_services_carbon_data.csv')
old_carbon.rename(columns={'value':'old_value'}, inplace=True)
old_carbon

,location_id,indicator,old_value
0,2346.0,SOC,0.000000e+00
1,2029.0,SOC,2.202742e+05
2,1915.0,SOC,4.076700e+02
3,3095.0,SOC,9.031557e+03
4,2707.0,SOC,9.816581e+03
...,...,...,...
207,3248.0,AGB,8.574929e+02
208,3582.0,AGB,9.770106e+05
209,1681.0,AGB,1.311131e+06
210,2339.0,AGB,2.919104e+02


In [44]:
df_merge = pd.merge(df_final, old_carbon, on = ['location_id', 'indicator'])
df_merge = df_merge[['iso','indicator','value', 'old_value']]
df_merge

,iso,indicator,value,old_value
0,ASM,AGB,8.291920e+01,0.000000e+00
1,AGO,AGB,1.105094e+05,3.080926e+04
2,AIA,AGB,1.101510e+01,6.524752e+00
3,ATG,AGB,2.714655e+03,1.078042e+03
4,ABW,AGB,1.856843e+02,1.374874e+03
...,...,...,...,...
201,VEN,SOC,1.468958e+06,4.366034e+06
202,VNM,SOC,8.665969e+06,5.100087e+06
203,VGB,SOC,6.823373e+03,2.306587e+02
204,VIR,SOC,1.410356e+04,1.609178e+03


In [46]:
df_merge.to_csv('../../../../data/old_new_carbon_data_check.csv', index=False)